# Test notebook for Fall 2023 DM Acceptance Test Campaign

This test will be executed on the LSST Science Platform Notebook Aspect, initialized with Science Pipelines release `w_2023_43`.

### Test case LVV-T97: Verify implementation of Uniqueness of IDs Across Data Releases
Verify that the IDs of Objects, Sources, DIAObjects, and DIASources from different Data Releases are unique.

In [1]:
# Confirm that the version of the Science Pipelines is v19_0_0:
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

jcarlin-nb
lsst_distrib          g4213664e8e+da93b84f60 	current w_2023_37 setup


In [2]:
import numpy as np
from lsst.daf.butler import Butler

We have processed `rc2_subset` (a small set of HSC data) two times. In the first, we changed lines 44-52 in `meas_base` [in _id_generator.py here](https://github.com/lsst/meas_base/blob/1bfaca56951770c88f4308da41de16d72ce40db9/python/lsst/meas/base/_id_generator.py#L44-L52) to read:

```
DEFAULT_RELEASE_ID = 8
"""Default release ID to embed in catalog IDs.

This can be changed globally to avoid having to override individual task
configs to set the release ID.
"""

DEFAULT_N_RELEASES = 10  # 1 means don't reserve space for releases.
"""Default number of releases to reserve space for in catalog IDs."""
```

For the second, we set `DEFAULT_RELEASE_ID = 9`. This should ensure that the two processing runs will not share any IDs in common.

In [3]:
repo = '/repo/main'
rc2_subset_collection_id8 = 'u/jcarlin/lvv-t97_8'
rc2_subset_collection_id9 = 'u/jcarlin/lvv-t97_9'

Initialize two butler repositories pointing to the collections for the two runs:

In [4]:
# Initialize the butler repo:
butler8 = Butler(repo, collections=rc2_subset_collection_id8)
butler9 = Butler(repo, collections=rc2_subset_collection_id9)

### Extract sourceTables from multiple processing runs and compare their IDs:

We select a random single visit for comparison.

In [5]:
visit = 30490

dataid_visit = {'visit':visit, 'instrument':'HSC'}

In [6]:
src_8 = butler8.get('sourceTable_visit', dataId=dataid_visit)
src_9 = butler9.get('sourceTable_visit', dataId=dataid_visit)

# Print the first few IDs to the screen:
for i in range(10):
    print(src_8.index[i], src_9.index[i])

4612504483635658753 5188965235939082241
4612504483635658754 5188965235939082242
4612504483635658755 5188965235939082243
4612504483635658756 5188965235939082244
4612504483635658757 5188965235939082245
4612504483635658758 5188965235939082246
4612504483635658759 5188965235939082247
4612504483635658760 5188965235939082248
4612504483635658761 5188965235939082249
4612504483635658762 5188965235939082250


Source IDs are the table indices (labeled "sourceId" above). We will extract those indices, and then compare them to check whether they share any elements in common.

In [7]:
ind_src8 = src_8.index
ind_src9 = src_9.index

Use the method `.isin` to check whether any elements of `ind_src8` are also in `ind_src9`. The result of this comparison is a boolean array, so by using the `numpy.sum` method, we can count how many elements of this array are True.

In [8]:
np.sum(ind_src8.isin(ind_src9))

0

The zero implies that there are no elements of `ind_src8` that are also in `ind_src9`, so we have demonstrated that the sourceIds from the two processing campaigns are unique.

### Extract objectTables from multiple processing runs and compare their IDs:

We select a random single patch for comparison. (Note that `rc2_subset` only contains data in tract 9813.)

In [9]:
tract = 9813
patch = 40

dataid_tract = {'tract':tract, 'patch':patch, 'skymap':'hsc_rings_v1', 'instrument':'HSC'}

In [10]:
obj_8 = butler8.get('objectTable', dataId=dataid_tract)
obj_9 = butler9.get('objectTable', dataId=dataid_tract)

obj_9

column,coord_dec,refBand,x,detect_isPatchInner,shape_xy,coord_ra,shape_yy,detect_isDeblendedSource,deblend_nChild,sky_object,...,z_kronFlux_flag,z_kronFlux_flag_bad_radius,z_kronFlux_flag_bad_shape,z_kronFlux_flag_bad_shape_no_psf,z_kronFlux_flag_edge,z_kronFlux_flag_no_fallback_radius,z_kronFlux_flag_no_minimum_radius,z_kronFlux_flag_small_radius,z_kronFlux_flag_used_minimum_radius,z_kronFlux_flag_used_psf_radius
objectId,,,,,,,,,,,,,,,,,,,,,
5406645294815248385,2.133690,i,15963.325496,False,1.939241,150.342998,4.838945,True,1,False,...,True,False,True,False,True,False,False,False,False,False
5406645294815248386,2.133776,i,16150.000000,False,-0.361829,150.334280,2.787343,True,1,False,...,True,False,True,False,False,False,False,False,False,False
5406645294815248387,2.134882,i,16367.552413,False,-0.393917,150.324121,3.554593,False,3,False,...,False,False,False,False,False,False,False,False,False,False
5406645294815248388,2.133683,i,16395.000000,False,-0.030194,150.322839,1.616722,True,1,False,...,True,False,True,False,True,False,False,False,False,False
5406645294815248389,2.133923,i,16707.893795,False,0.386806,150.308227,7.236080,False,6,False,...,True,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5406645294815297873,2.321529,g,19987.829656,True,NaN,150.155046,NaN,False,0,False,...,True,False,True,False,False,False,False,False,False,False
5406645294815297874,2.324527,g,19779.226432,True,NaN,150.164788,NaN,False,0,False,...,True,False,True,False,False,False,False,False,False,False
5406645294815297875,2.324735,g,19979.000000,False,-3.420993,150.155458,2.915249,False,0,False,...,True,False,True,False,False,False,False,False,False,False


In [11]:
print(len(obj_8), len(obj_9))

33981 34812


Object IDs are the table indices (labeled "objectId" above). We will extract those indices, and then compare them to check whether they share any elements in common.

In [12]:
ind_obj8 = obj_8.index
ind_obj9 = obj_9.index

In [13]:
np.sum(ind_obj8.isin(ind_obj9))

0

The zero implies that there are no elements of `ind_obj8` that are also in `ind_obj9`, so we have demonstrated that the objectIds from the two processing campaigns are unique.